# Center-TBI statistics (from XNAT web interface export)
Short description to add here...

Please edit login.cfg with your credentials before executing this script.

Before (re-)running this script, please clear output, shutdown and relaunch kernel, close down and reopen your browser, and then (re-)launch all the cells! Else the memory is not correctly freed (this is a bug in ipywidgets or jupyter notebook).


In [ ]:
%load_ext autoreload
%autoreload 2
# BEWARE: autoreload works on functions and on general code, but NOT on new class methods:
# if you add or change the name of a method, you have to reload the kernel!
# also it will fail if you use super() calls in the classes you change

# Profilers:
# http://pynash.org/2013/03/06/timing-and-profiling/
# http://mortada.net/easily-profile-python-code-in-jupyter.html
# use %lprun -m module func(*args, **kwargs)
%load_ext line_profiler
%load_ext memory_profiler

import json
import pandas as pd
from fdict import fdict

# Setup some display options for pandas
pd.set_option('max_columns', 40)
pd.set_option('expand_frame_repr', False)

## AUX FUNCTIONS

## Parameters

In [ ]:
# Edit the filepath to the csv files here
subjects_filepath = 'db/lrq3000_8_6_2017_5_39_9-subjects.csv'
mrisessions_filepath = 'db/lrq3000_8_6_2017_5_40_41-mrsessions.csv'

## The real stuff!

In [ ]:
# FIRST STEP: need to be able to decode the malformed json fields
# «Complex objects may be returned as encapsulated JSON strings», from dHCP XNAT fork code
import re
def fix_xnat_json(badjson):
    '''A simple state machine to fix missing delimiters/punctuation in a XNAT malformatted JSON string
    This is not very robust against nested structures, for example a list of objects will fail here.
    TODO: we should turn flag* vars into a list or a dict, so we can maintain the state of flags per nesting level.'''
    corrected = []
    level = 0  # store the current object nested level
    flaglist = False  # are we in the middle of a list?
    flagkeyprev = False  # did we have a key just before, and so now we are in the value?
    flagkeystart = False # are we inside a key string?
    flagstring = False  # are we in the middle of a string?
    flagregister = True  # should we register the current character or skip it in the end result?
    flagadddelimiter = False  # should we add a delimiter after the current character?
    flagprependdelimiter = False # should we prepend a delimiter before current character?
    flagvalue = False  # are we in a non-string value (eg, boolean true/false or number)
    flagsyntax = False  # did we meet a syntax symbol (excluding ")

    # Walk through each character of the input string (this is a simple state machine)
    for c in badjson:
        if c == '{' and not flagstring:
            # Beginning of an object, clean up previous state and increase nesting level
            level += 1
            flagadddelimiter = False
            flagkeyprev = False
            flaglist = False
        elif c == '}' and not flagstring:
            # Object end, we should add a delimiter and decrease nesting level
            level -= 1
            flagkeyprev = False
            if level > 0:
                # do not add the delimiter after if it is the last/root object
                flagadddelimiter = True
                if flagvalue:
                    flagvalue = False
            flagsyntax = True
        elif c == '[' and not flagstring:
            # Beginning of a list, clean up previous state
            flaglist = True
            flagadddelimiter = False
            flagkeyprev = False
            flagsyntax = True
        elif c == ']' and not flagstring:
            # List end, we should add a delimiter
            flaglist = False
            flagkeyprev = False
            flagadddelimiter = True
            if flagvalue:
                flagvalue = False
            flagsyntax = True
        elif c == '"':
            # Beginning or ending of a string
            if flagstring and (flagkeyprev or flaglist):
                # Ending of a string, and of a value string, we need to add a delimiter
                flagkeyprev = False
                #flagvalue = True
                flagadddelimiter = True
            elif not flagstring and not (flagkeyprev or flaglist):
                # No key previous to this string and not in a list, so this is probably a key string
                flagkeystart = True
            elif flagvalue:
                # There was a non-string value just before and now it ended, we need to prepend a delimiter
                flagvalue = False
                flagkeyprev = False
                flagprependdelimiter = True
            # Switch on and off the flag
            flagstring = not flagstring
        elif c == ':' and not flagstring:
            # Separator between key and values, we flag now that we had a key just before
            if not flagstring and not flaglist and not flagkeyprev:
                flagkeyprev = True
                flagkeystart = False
            flagsyntax = False
        elif c == ' ':
            if (flagstring and flagkeystart) or flagvalue:
                # Space inside a key string or a non-string value, we remove (keys should NOT have any space! Values can)
                flagregister = False
            #elif flagvalue:
                # We just finished processing a non-string value, we add a delimiter
                # Deprecated, because some values might have spaces in the middle...
                #flagvalue = False
                #flagkeyprev = False
                #flagregister = False
                #flagadddelimiter = True
        elif c == ',' and not flagstring:
            if not flagstring:
                flagadddelimiter = False
                flagkeyprev = False
                flagvalue = False
        elif not flagstring:
            # Any other character is probably a value (number, boolean, etc)
            if not flagvalue:
                flagvalue = True

        if flagsyntax:
            # Manage values when a syntax symbol is met
            flagsyntax = False
            if flagvalue:
                # If we were in a value just before, we should add a delimiter
                flagadddelimiter = True
            flagvalue = False
            flagkeyprev = False

        if flagprependdelimiter:
            flagprependdelimiter = False
            corrected += ','

        if flagregister:
            corrected += c
        else:
            flagregister = True

        if flagadddelimiter:
            flagadddelimiter = False
            corrected += ','
    # Check that all tags were closed
    if level != 0:
        raise ValueError('ERROR: malformed JSON, at least one object is not closed!')
    if flaglist:
        raise ValueError('ERROR: malformed JSON, at least one list was not closed!')
    if flagstring:
        raise ValueError('ERROR: malformed JSON, at least one string was not closed!')
    # Convert result list to a string
    corrected_str = ''.join(corrected)
    # Remove useless comma (ie, at the end of a list or after the last field of an object, just before the closing tag)
    corrected_str = re.sub(r',\s*([}\]])', r'\1', corrected_str)
    # Remove superfluous spaces (just for pleasing the eye)
    corrected_str = re.sub(r'\s+', r' ', corrected_str)  # strip multiple spaces into only one
    # Remove duplicated delimiters
    corrected_str = re.sub(r',+', r',', corrected_str)
    # Remove end delimiter of root object
    corrected_str = re.sub(r'}(,+)\s*$', r'}', corrected_str)
    # Return the result
    return corrected_str

# Unit test
badjson = '''{"pipeline": "tbi_mri_processin  g"  "software_versions": {"msmetrix-dev": {"tags": ["icobrain_1.3.2" "lala" "lolo" ]    "commit_hash": "9880242"  "branch": "master"  "commit_date": "Tue Jul 12 08:13:02 2016 +0000"}  "center-tbi": {"tags": ["ctbi_pre_pipeline_2.0-123-g52bbb17" ]    "commit_hash": "52bbb17"  "branch": "master"  "commit_date": "Tue Jul 12 08:16:41 2016 +0000"}}  "numpy": "1.9.2"  "nibabel": "2.0.1"}'''
assert fix_xnat_json(badjson) == '''{"pipeline": "tbi_mri_processin g", "software_versions": {"msmetrix-dev": {"tags": ["icobrain_1.3.2", "lala", "lolo"], "commit_hash": "9880242", "branch": "master", "commit_date": "Tue Jul 12 08:13:02 2016 +0000"}, "center-tbi": {"tags": ["ctbi_pre_pipeline_2.0-123-g52bbb17"], "commit_hash": "52bbb17", "branch": "master", "commit_date": "Tue Jul 12 08:16:41 2016 +0000"}}, "numpy": "1.9.2", "nibabel": "2.0.1"}'''
assert json.loads(fix_xnat_json(badjson))

badjson2 = '''{"a": "b" "c": {"d": 0.9 "e": true "f": false "h": ["5" "6" "7"]}, "g": ["1" "2" "3"]}'''
assert fix_xnat_json(badjson2) == '''{"a": "b", "c": {"d": 0.9,"e": true,"f": false,"h": ["5", "6", "7"]}, "g": ["1", "2", "3"]}'''
assert json.loads(fix_xnat_json(badjson2))

badjson3 = '''{"ghosting_calculation": {"process_types": ["T1"  "T2"  "FLAIR"  "SWI" ]    "mask_ap": "qc_templates/maskAP.nii.gz"  "scan_type_order": ["T1"  "FLAIR"  "T2" ]    "mask_side": "qc_templates/maskSide.nii.gz"}  "segmentation": {"scan_type_order": ["T1"  "FLAIR"  "T2" ]    "brain_mask": "qc_templates/wbrainmask.nii.gz"}  "dicom2nifti": {"process_types": ["T1"  "T2"  "FLAIR"  "SWI"  "DTI"  "b0"  "rs_fMRI" ]    "deface": true  "deface_types": ["T1"  "T2"  "FLAIR"  "SWI"  "T2*" ]    "overwrite": false}  "tbi_dti_qc": {"check_gradient_table": {"process_types": ["DTI" ]  }  "dti_tensor_residual_calculation": {"process_types": ["DTI" ]  }  "temporal_snr": {"process_types": ["DTI" ]    "brain_mask": "qc_templates/wbrainmask.nii.gz"}  "percentage_signal_change": {"process_types": ["DTI" ]    "brain_mask": "qc_templates/wbrainmask.nii.gz"  "mask_threshold": 0.9}  "input_data": {"process_types": ["DTI" ]    "data_types": ["NIFTI"  "BVAL"  "BVEC" ]  }  "fa_md_feature_estimation": {"process_types": ["DTI" ]  }}  "dicom_qc": {"process_types": ["T2"  "T1"  "FLAIR"  "DTI"  "SWI"  "rs_fMRI" ]    "center": "dicom_qc"  "server_name": "neuro-imaging.center-tbi.eu"}  "smoothed_image_statistics": {"process_types": ["T1"  "T2"  "FLAIR" ]  }  "cnr_calculation": {"process_types": ["T1"  "T2"  "FLAIR"  "DTI" ]    "noise_mask": "qc_templates/maskNoise.nii.gz"  "mask_threshold": 0.9}  "scan_type_translation": {"server_name": "neuro-imaging.center-tbi.eu"}  "input_data": {"proce ss_types": ["T1"  "T2"  "FLAIR"  "SWI"  "DTI"  "b0"  "rs_fMRI" ]    "data_types": ["DICOM"  "NIFTI"  "BVAL"  "BVEC" ]  }  "tbi_report": {}  "tbi_fmri_qc": {"fmri_motion_detection": {"process_types": [  "rs_fMRI" ]    "translation_threshold": 3  "rotation_threshold": 3}  "input_data": {"process_types": ["rs_fMRI" ]    "data_types": ["NIFTI" ]  }  "percentage_signal_change": {"process_types": ["rs_fMRI" ]    "brain_mask": "qc_templates/wbrainmask.nii.gz"  "mask_threshold": 0.9}  "temporal_snr": {"process_types": ["rs_fMRI" ]    "brain_mask": "qc_templates/wbrainmask.nii.gz"}  "fmri_snr_calculation": {"process_types": ["rs_fMRI" ]  }}  "head_coverage": {"process_types": ["T1"  "T2"  "FLAIR"  "SWI"  "DTI"  "b0"  "rs_FMRI" ]    "atlas_mask": "NormalSharpAtlas/mni_icbm152_nlin_asym_09a/mni_icbm152_t1_tal_nlin_asym_09a_mask.nii.gz"}  "registration": {"process_types": ["T1"  "T2"  "FLAIR"  "SWI"  "DTI"  "b0"  "rs_fMRI" ]    "scan_type_order": ["T1"  "FLAIR"  "T2" ]  }  "gm_wm_statistics": {"process_types": ["T1"  "T2"  "FLAIR" ]  }  "snr_calculation": {"process_types": ["T1"  "T2"  "FLAIR"  "DTI" ]    "noise_mask": "qc_templates/maskNoise.nii.gz"  "mask_types": {"SWI": "CSF"  "DTI": "CSF"  "T2": "CSF"  "T1": "WM"  "B0": "CSF"  "FLAIR": "GM"}  "mask_threshold": 0.9}  "upload_results": {}  "protocol_check": {"validate_order": true  "sequence": ["T2"  "T1"  "FLAIR"  "SWI"  "DTI"  "DTI"  "rs_fMRI" ]  }}'''
assert fix_xnat_json(badjson3) == '''{"ghosting_calculation": {"process_types": ["T1", "T2", "FLAIR", "SWI"], "mask_ap": "qc_templates/maskAP.nii.gz", "scan_type_order": ["T1", "FLAIR", "T2"], "mask_side": "qc_templates/maskSide.nii.gz"}, "segmentation": {"scan_type_order": ["T1", "FLAIR", "T2"], "brain_mask": "qc_templates/wbrainmask.nii.gz"}, "dicom2nifti": {"process_types": ["T1", "T2", "FLAIR", "SWI", "DTI", "b0", "rs_fMRI"], "deface": true,"deface_types": ["T1", "T2", "FLAIR", "SWI", "T2*"], "overwrite": false}, "tbi_dti_qc": {"check_gradient_table": {"process_types": ["DTI"]}, "dti_tensor_residual_calculation": {"process_types": ["DTI"]}, "temporal_snr": {"process_types": ["DTI"], "brain_mask": "qc_templates/wbrainmask.nii.gz"}, "percentage_signal_change": {"process_types": ["DTI"], "brain_mask": "qc_templates/wbrainmask.nii.gz", "mask_threshold": 0.9}, "input_data": {"process_types": ["DTI"], "data_types": ["NIFTI", "BVAL", "BVEC"]}, "fa_md_feature_estimation": {"process_types": ["DTI"]}}, "dicom_qc": {"process_types": ["T2", "T1", "FLAIR", "DTI", "SWI", "rs_fMRI"], "center": "dicom_qc", "server_name": "neuro-imaging.center-tbi.eu"}, "smoothed_image_statistics": {"process_types": ["T1", "T2", "FLAIR"]}, "cnr_calculation": {"process_types": ["T1", "T2", "FLAIR", "DTI"], "noise_mask": "qc_templates/maskNoise.nii.gz", "mask_threshold": 0.9}, "scan_type_translation": {"server_name": "neuro-imaging.center-tbi.eu"}, "input_data": {"process_types": ["T1", "T2", "FLAIR", "SWI", "DTI", "b0", "rs_fMRI"], "data_types": ["DICOM", "NIFTI", "BVAL", "BVEC"]}, "tbi_report": {}, "tbi_fmri_qc": {"fmri_motion_detection": {"process_types": [ "rs_fMRI"], "translation_threshold": 3,"rotation_threshold": 3}, "input_data": {"process_types": ["rs_fMRI"], "data_types": ["NIFTI"]}, "percentage_signal_change": {"process_types": ["rs_fMRI"], "brain_mask": "qc_templates/wbrainmask.nii.gz", "mask_threshold": 0.9}, "temporal_snr": {"process_types": ["rs_fMRI"], "brain_mask": "qc_templates/wbrainmask.nii.gz"}, "fmri_snr_calculation": {"process_types": ["rs_fMRI"]}}, "head_coverage": {"process_types": ["T1", "T2", "FLAIR", "SWI", "DTI", "b0", "rs_FMRI"], "atlas_mask": "NormalSharpAtlas/mni_icbm152_nlin_asym_09a/mni_icbm152_t1_tal_nlin_asym_09a_mask.nii.gz"}, "registration": {"process_types": ["T1", "T2", "FLAIR", "SWI", "DTI", "b0", "rs_fMRI"], "scan_type_order": ["T1", "FLAIR", "T2"]}, "gm_wm_statistics": {"process_types": ["T1", "T2", "FLAIR"]}, "snr_calculation": {"process_types": ["T1", "T2", "FLAIR", "DTI"], "noise_mask": "qc_templates/maskNoise.nii.gz", "mask_types": {"SWI": "CSF", "DTI": "CSF", "T2": "CSF", "T1": "WM", "B0": "CSF", "FLAIR": "GM"}, "mask_threshold": 0.9}, "upload_results": {}, "protocol_check": {"validate_order": true,"sequence": ["T2", "T1", "FLAIR", "SWI", "DTI", "DTI", "rs_fMRI"]}}'''
assert json.loads(fix_xnat_json(badjson3))

badjson4 = '''{"ghosting_calculation": {"process_types": ["T1"  "T2"  "FLAIR"  "SWI" ]    "mask_ap": "qc_templates/maskAP.nii.gz"  "scan_type_order": ["T1"  "FLAIR"  "T2" ]    "mask_side": "qc_templates/maskSide.nii.gz"}  "segmentation": {"scan_type_order": ["T1"  "FLAIR"  "T2" ]    "brain_mask": "qc_templates/wbrainmask.nii.gz"}  "dicom2nifti": {"process_types": ["T1"  "T2"  "FLAIR"  "SWI"  "DTI"  "b0"  "rs_fMRI" ]    "deface": true  "deface_types": ["T1"  "T2"  "FLAIR"  "SWI"  "T2*" ]    "overwrite": false}  "tbi_dti_qc": {"check_gradient_table": {"process_types": ["DTI" ]  }}}'''
assert fix_xnat_json(badjson4) == '''{"ghosting_calculation": {"process_types": ["T1", "T2", "FLAIR", "SWI"], "mask_ap": "qc_templates/maskAP.nii.gz", "scan_type_order": ["T1", "FLAIR", "T2"], "mask_side": "qc_templates/maskSide.nii.gz"}, "segmentation": {"scan_type_order": ["T1", "FLAIR", "T2"], "brain_mask": "qc_templates/wbrainmask.nii.gz"}, "dicom2nifti": {"process_types": ["T1", "T2", "FLAIR", "SWI", "DTI", "b0", "rs_fMRI"], "deface": true,"deface_types": ["T1", "T2", "FLAIR", "SWI", "T2*"], "overwrite": false}, "tbi_dti_qc": {"check_gradient_table": {"process_types": ["DTI"]}}}'''
assert json.loads(fix_xnat_json(badjson4))

badjson5 = '''{"ghosting_calculation": {"process_types": ["T1"  "T2"  "FLAIR"  "SWI" ]    "mask_ap": "qc_templates/maskAP.nii.gz"  "scan_type_order": ["T1"  "FLAIR"  "T2" ]    "mask_side": "qc_templates/maskSide.nii.gz"}  "segmentation": {"scan_type_order": ["T1"  "FLAIR"  "T2" ]    "brain_mask": "qc_templates/wbrainmask.nii.gz"}  "dicom2nifti": {"process_types": ["T1"  "T2"  "FLAIR"  "SWI"  "DTI"  "b0"  "rs_fMRI"  "Fieldmap" ]    "deface": true  "deface_types": ["T1"  "T2"  "FLAIR "  "SWI"  "T2*" ]    "overwrite": false}  "tbi_dti_qc": {"check_gradient_table": {"process_types": ["DTI" ]  }  "dti_tensor_residual_calculation": {"process_types": ["DTI" ]    "atlas_mask": nu  ll}  "temporal_snr": {"process_types": ["DTI" ]    "brain_mask": "qc_templates/wbrainmask.nii.gz"}  "percentage_signal_change": {"process_types": ["DTI" ]    "brain_mask": "qc_templates/wbrainmask.nii.gz"  "mask_threshold": 0.9}  "input_data": {"process_types": ["DTI" ]    "data_types": ["NIFTI"  "BVAL"  "BVEC" ]  }  "fa_md_feature_estimation": {"process_types": ["DTI" ]  }}  "dicom_qc": {"process_types": ["T2"  "T1"  "FLAIR"  "DTI"  "SWI"  "rs_fMRI" ]    "center": "dicom_qc"  "server_name": "neuro-imaging.center-tbi.eu"}  "smoothed_image_statistics": {"process_types": ["T1"  "T2"  "FLAIR" ]  }  "cnr_calculation": {"process_types": ["T1"  "T2"  "FLAIR"  "DTI" ]    "noise_mask": "qc_templates/maskNoise.nii.gz"  "mask_threshold": 0.9}  "scan_type_translation": {"server_name": "neuro-imaging.center-tbi.eu"}  "input_data": {"process_types": ["T1"  "T2"  "FLAIR"  "SWI"  "DTI"  "b0"  "rs_fMRI" ]    "data_types": ["DICOM"  "NIFTI"  "BVAL"  "BVEC" ]  }  "tbi_report": {}  "tbi_fmri_qc": {"fmri_motion_detection": {"process_types": ["rs_fMRI" ]    "translation_threshold": 3  "rotation_threshold": 3}  "input_data": {"process_types": ["rs_fMRI" ]    "data_types": ["NIFTI" ]  }  "percentage_signal_change": {"process_types": ["rs_fMRI" ]    "brain_mask": "qc_templates/wbrainmask.nii.gz"  "mask_threshold": 0.9}  "temporal_snr": {"process_types": ["rs_fMRI" ]    "brain_mask": "qc_templates/wbrainmask.nii.gz"}  "fmri_snr_calculation": {"process_types": ["rs_fMRI" ]  }}  "head_coverage": {"process_types": ["T1"  "T2"  "FLAIR"  "SWI"  "DTI"  "b0"  "rs_FMRI" ]    "atlas_mask": "NormalSharpAtlas/mni_icbm152_nlin_asym_09a/mni_icbm152_t1_tal_nlin_asym_09a_mask.nii.gz"}  "registration": {"process_types": ["T1"  "T2"  "FLAIR"  "SWI"  "DTI"  "b0"  "rs_fMRI" ]    "scan_type_order": ["T1"  "FLAIR"  "T2" ]  }  "gm_wm_statistics": {"process_types": ["T1"  "T2"  "FLAIR" ]  }   "snr_calculation": {"process_types": ["T1"  "T2"  "FLAIR"  "DTI" ]    "noise_mask": "qc_templates/maskNoise.nii.gz"  "mask_types": {"SWI": "CSF"  "DTI": "CSF"  "T2": "CSF"  "T1": "WM"  "B0": "CSF"  "FLAIR": "GM"}  "mask_threshold": 0.  9}  "upload_results": {}  "protocol_check": {"validate_order": true  "sequence": ["T2"  "T1"  "FLAIR"  "SWI"  "DTI"  "DTI"  "rs_fMRI" ]  }}'''
assert fix_xnat_json(badjson5) == '''{"ghosting_calculation": {"process_types": ["T1", "T2", "FLAIR", "SWI"], "mask_ap": "qc_templates/maskAP.nii.gz", "scan_type_order": ["T1", "FLAIR", "T2"], "mask_side": "qc_templates/maskSide.nii.gz"}, "segmentation": {"scan_type_order": ["T1", "FLAIR", "T2"], "brain_mask": "qc_templates/wbrainmask.nii.gz"}, "dicom2nifti": {"process_types": ["T1", "T2", "FLAIR", "SWI", "DTI", "b0", "rs_fMRI", "Fieldmap"], "deface": true,"deface_types": ["T1", "T2", "FLAIR ", "SWI", "T2*"], "overwrite": false}, "tbi_dti_qc": {"check_gradient_table": {"process_types": ["DTI"]}, "dti_tensor_residual_calculation": {"process_types": ["DTI"], "atlas_mask": null}, "temporal_snr": {"process_types": ["DTI"], "brain_mask": "qc_templates/wbrainmask.nii.gz"}, "percentage_signal_change": {"process_types": ["DTI"], "brain_mask": "qc_templates/wbrainmask.nii.gz", "mask_threshold": 0.9}, "input_data": {"process_types": ["DTI"], "data_types": ["NIFTI", "BVAL", "BVEC"]}, "fa_md_feature_estimation": {"process_types": ["DTI"]}}, "dicom_qc": {"process_types": ["T2", "T1", "FLAIR", "DTI", "SWI", "rs_fMRI"], "center": "dicom_qc", "server_name": "neuro-imaging.center-tbi.eu"}, "smoothed_image_statistics": {"process_types": ["T1", "T2", "FLAIR"]}, "cnr_calculation": {"process_types": ["T1", "T2", "FLAIR", "DTI"], "noise_mask": "qc_templates/maskNoise.nii.gz", "mask_threshold": 0.9}, "scan_type_translation": {"server_name": "neuro-imaging.center-tbi.eu"}, "input_data": {"process_types": ["T1", "T2", "FLAIR", "SWI", "DTI", "b0", "rs_fMRI"], "data_types": ["DICOM", "NIFTI", "BVAL", "BVEC"]}, "tbi_report": {}, "tbi_fmri_qc": {"fmri_motion_detection": {"process_types": ["rs_fMRI"], "translation_threshold": 3,"rotation_threshold": 3}, "input_data": {"process_types": ["rs_fMRI"], "data_types": ["NIFTI"]}, "percentage_signal_change": {"process_types": ["rs_fMRI"], "brain_mask": "qc_templates/wbrainmask.nii.gz", "mask_threshold": 0.9}, "temporal_snr": {"process_types": ["rs_fMRI"], "brain_mask": "qc_templates/wbrainmask.nii.gz"}, "fmri_snr_calculation": {"process_types": ["rs_fMRI"]}}, "head_coverage": {"process_types": ["T1", "T2", "FLAIR", "SWI", "DTI", "b0", "rs_FMRI"], "atlas_mask": "NormalSharpAtlas/mni_icbm152_nlin_asym_09a/mni_icbm152_t1_tal_nlin_asym_09a_mask.nii.gz"}, "registration": {"process_types": ["T1", "T2", "FLAIR", "SWI", "DTI", "b0", "rs_fMRI"], "scan_type_order": ["T1", "FLAIR", "T2"]}, "gm_wm_statistics": {"process_types": ["T1", "T2", "FLAIR"]}, "snr_calculation": {"process_types": ["T1", "T2", "FLAIR", "DTI"], "noise_mask": "qc_templates/maskNoise.nii.gz", "mask_types": {"SWI": "CSF", "DTI": "CSF", "T2": "CSF", "T1": "WM", "B0": "CSF", "FLAIR": "GM"}, "mask_threshold": 0.9}, "upload_results": {}, "protocol_check": {"validate_order": true,"sequence": ["T2", "T1", "FLAIR", "SWI", "DTI", "DTI", "rs_fMRI"]}}'''
assert json.loads(fix_xnat_json(badjson5))

badjson6 = '''{"lal a": nu   ll}'''
assert fix_xnat_json(badjson6) == '{"lala": null}'
assert json.loads(fix_xnat_json(badjson6))

badjson7 = '''{"11": [{"dicom_field_expected_value": "31" "dicom_file_value": "31"} {"lala": true "lolo": nu  ll}]}'''
assert fix_xnat_json(badjson7) == '''{"11": [{"dicom_field_expected_value": "31", "dicom_file_value": "31"}, {"lala": true,"lolo": null}]}'''
assert json.loads(fix_xnat_json(badjson7))

In [ ]:
badjson7 = '''{"ghosting_calculation": {"8": 0.7034861380159364  "3": 0.84972068521977484  "2": 1.1356784846741803  "4": 1.3192496272090475}  "segmentation": {"3": ["GM.nii.gz"  "WM.nii.gz"  "CSF.nii.gz" ]  }  "dicom2nifti": {"8": {"defacing_mask": "8_deface_mask.nii.gz"}  "3": {"defacing_mask": "3_deface_mask.nii.gz"}  "2": {"defacing_mask": "2_deface_mask.nii.gz"}  "4": {"defacing_mask": "4_deface_mask.nii.gz"}}  "tbi_dti_qc": {"fa_md_feature_estimation": {"9": {"fa_value": 0.50402228993967613  "md_value": 0.00075898986543204326}  "11": {"fa_value": 0.50312148745008711  "md_value": 0.0007616720948551382}}  "chec k_gradient_table": {"9": {"number_of_bvalues": 1  "number_of_b0": 1  "directions_per_bvalue": {"1000": 30}}  "11": {"number_of_bvalues": 1  "number_of_b0": 1  "directions_per_bvalue": {"1000": 30}}}  "temporal_snr": {"9  ": {"temporal_snr_image": "9_temporal_snr.nii.gz"  "temporal_snr_value": 6.6750913111902586  "standard_deviation_image": "9_temporal_snr.nii.gz"  "mean_image": "9_temporal_snr.nii.gz"}  "11": {"temporal_snr_image": "11_temporal_snr.nii.gz"  "temporal_snr_value": 6.9431868083366624  "standard_deviation_image": "11_temporal_snr.nii.gz"  "mean_image": "11_temporal_snr.nii.gz"}}  "percentage_signal_change": {"9": {"psc_mean": 0.46415164643697843  "pcs_std": 0.0048018485298684261}  "11": {"psc_mean": 0.46506220528464781  "pcs_std": 0.0040992226719628192}}  "dti_tensor_residual_calculation": {"9": {"residual_png": "9_tensor_residuals.png"  "fa_map": "9_fa_map.nii.gz"  "misalignment_nii": "9_misalignment.nii.gz"  "outlier_order": [13  10  1  12  7  14  9  8  15  11  6  4  3  5  2  0 ]    "number_of_outliers": [0  2602  1902  2103  2208  1955  2242  2505  2365  2376  3093  2260  2516  3312  2405  2271 ]    "fa_hist_png": "9_fa_hist.png"  "MEANVOX": 2291.0  "md_map": "9_md_map.nii.gz"  "MAXVOX": 3312  "residual_nii": "9_tensor_residuals.nii.gz"}  "11": {"residual_png": "11_tensor_residuals.png"  "fa_map": "11_fa_map.nii.gz"  "misalignment_nii": "11_misalignment.nii.gz"  "outlier_order": [1  13  10  9  7  14  6  3  15  8  4  11  12  5  2  0 ]    "number_of_outliers": [0  3663  2035  2297  2203  2051  2323  2503  2263  2545  2755  2176  2114  3011  2442  2278 ]    "fa_hist_png": "11_fa_hist.png"  "MEANVOX": 2293.0967741 935483  "md_map": "11_md_map.nii.gz"  "MAXVOX": 3663  "residual_nii": "11_tensor_residuals.nii.gz"}}}  "dicom_qc": {"11": [{"dicom_field_expected_value": "31"  "dicom_file_value": "31"  "dicom_tag": ""  "flag_value": 1  "dicom_file_id": 2  "dicom_field_name": "frames"}  {"dicom_field_expected_value": "2"  "dicom_file_value": "2.0"  "dicom_tag": "(0018  0050)"  "flag_value": 1  "dicom_file_id": 2  "dicom_field_name": "Slice Thickness"}  {"dicom_field_expected_value": "11600"  "dicom_file_value": "11600.0"  "dicom_tag": "(0018  0080)"  "flag_value": 1  "dicom_file_id": 2  "dicom_field_name": "Repetition Time"}  {"dicom_fie  ld_expected_value": "91"  "dicom_file_value": "91.0"  "dicom_tag": "(0018  0081)"  "flag_value": 1  "dicom_file_id": 2  "dicom_field_name": "Echo Time"}  {"dicom_field_expected_value": "1700"  "dicom_file_value": "1700.0"  "dicom_tag": "(0018  0095)"  "flag_value": 1  "dicom_file_id": 2  "dicom_field_name": "Pixel Bandwidth"}  {"dicom_field_expected_value": "[128  0  0  128 ]  "  "dicom_file_value": "[128  0  0  128 ]  "  "dicom_tag": "(0018  1310)"  "flag_value": 1  "dicom_file_id": 2  "dicom_field_name": "Acquisition Matrix"}  {"dicom_field_expected_value": "90"  "dicom_file_value": "90.0"  "dicom_tag": "(0018  1314)"  "flag_value": 1  "dicom_file_id": 2  "dicom_field_name": "Flip Angle"}  {"dicom_field_expected_value": "1152"  "dicom_file_value": "1152"  "dicom_tag": "(0028  0010)"  "flag_value": 1  "dicom_file_id": 2  "dicom_field_name": "Rows"}  {"dicom_field_expected_value": "1152"  "dicom_file_value": "1152"  "dicom_tag": "(0028  0011)"  "flag_value": 1  "dicom_file_id": 2  "dicom_field_name": "Columns"}  {"dicom_field_expected_value": "['2'  '2' ]  "  "dicom_file_value": "['2'  '2' ]  "  "dicom_tag": "(0028  0030)"  "flag_value": 1  "dicom_file_id": 2  "dicom_field_name": "Pixel Spacing"} ]    "14": [{"dicom_field_expected_value": "167"  "dicom_file_value": "167"  "dicom_tag": ""  "flag_value": 1  "dicom_file_id": 77  "dicom_field_name": "frames"}  {"dicom_field_expected_value": "3"  "dicom_fi le_value": "3.0"  "dicom_tag": "(0018  0050)"  "flag_value": 1  "dicom_file_id": 77  "dicom_field_name": "Slice Thickness"}  {"dicom_field_expected_value": "2480"  "dicom_file_value": "2480.0"  "dicom_tag": "(0018  0080)"  "flag_value": 1  "dicom_file_id": 77  "dicom_field_name": "Repetition Time"}  {"dicom_field_expected_value": "28"  "dicom_file_value": "28.0"  "dicom_tag": "(0018  0081)"  "flag_value": 1  "dicom_file_id": 77  "dicom_field_name": "Echo Time"}  {"dicom_field_expected_value": "2230"  "dicom_file_value": "2230.0"  "dicom_tag": "(0018  0095)"  "flag_value": 1  "dicom_file_id": 77  "dicom_field_name": "Pixel Bandwidth  "}  {"dicom_field_expected_value": "[64  0  0  64 ]  "  "dicom_file_value": "[64  0  0  64 ]  "  "dicom_tag": "(0018  1310)"  "flag_value": 1  "dicom_file_id": 77  "dicom_field_name": "Acquisition Matrix"}  {"dicom_field_expected_value": "70"  "dicom_file_value": "70.0"  "dicom_tag": "(0018  1314)"  "flag_value": 1  "dicom_file_id": 77  "dicom_field_name": "Flip Angle"}  {"dicom_field_expected_value": "448"  "dicom_file_value": "448"  "dicom_tag": "(0028  0010)"  "flag_value": 1  "dicom_file_id": 77  "dicom_field_name": "Rows"}  {"dicom_field_expected_value": "448"  "dicom_file_value": "448"  "dicom_tag": "(0028  0011)"  "flag_value": 1  "dicom_file_id": 77  "dicom_field_name": "Columns"}  {"dicom_field_expected_value": "['3'  '3' ]  "  "dicom_file_value": "['3'  '3' ]  "  "dicom_tag": "(0028  0030)"  "flag_value": 1  "dicom_file_id": 77  "dicom_field_name": "Pixel Spacing"} ]    "3": [{"dicom_field_expected_value": "192"  "dicom_file_value": "192"  "dicom_tag": ""  "flag_value": 1  "dicom_file_id": 94  "dicom_field_name": "frames"}  {"dicom_field_expected_value": "1"  "dicom_file_value": "1.0"  "dicom_tag": "(0018  0050)"  "flag_value": 1  "dicom_file_id": 94  "dicom_field_name": "Slice Thickness"}  {"dicom_field_expected_value": "2300"  "dicom_file_value": "2300.0"  "dicom_tag": "(0018  0080)"  "flag_value": 1  "dicom_file_id": 94  "dicom_field_name": "Repetition Time"}  {"dicom_field_exp ected_value": "2.98"  "dicom_file_value": "2.98"  "dicom_tag": "(0018  0081)"  "flag_value": 1  "dicom_file_id": 94  "dicom_field_name": "Echo Time"}  {"dicom_field_expected_value": "900"  "dicom_file_value": "900.0"  "dicom_tag": "(0018  0082)"  "flag_value": 1  "dicom_file_id": 94  "dicom_field_name": "Inversion Time"}  {"dicom_field_expected_value": "240"  "dicom_file_value": "240.0"  "dicom_tag": "(0018  0095)"  "flag_value": 1  "dicom_file_id": 94  "dicom_field_name": "Pixel Bandwidth"}  {"dicom_field_expected_value": "[0  256  256  0 ]  "  "dicom_file_value": "[0  256  256  0 ]  "  "dicom_tag": "(0018  1310)"  "flag_value": 1  "dicom_file_id"  : 94  "dicom_field_name": "Acquisition Matrix"}  {"dicom_field_expected_value": "9"  "dicom_file_value": "9.0"  "dicom_tag": "(0018  1314)"  "flag_value": 1  "dicom_file_id": 94  "dicom_field_name": "Flip Angle"}  {"dicom_field_expected_value": "256"  "dicom_file_value": "256"  "dicom_tag": "(0028  0010)"  "flag_value": 1  "dicom_file_id": 94  "dicom_field_name": "Rows"}  {"dicom_field_expected_value": "256"  "dicom_file_value": "256"  "dicom_tag": "(0028  0011)"  "flag_value": 1  "dicom_file_id": 94  "dicom_field_name": "Columns"}  {"dicom_field_expected_value": "['1'  '1' ]  "  "dicom_file_value": "['1'  '1' ]  "  "dicom_tag": "(0028  0030)"  "flag_value": 1  "dicom_file_id": 94  "dicom_field_name": "Pixel Spacing"} ]    "2": [{"dicom_field_expected_value": "192"  "dicom_file_value": "192"  "dicom_tag": ""  "flag_value": 1  "dicom_file_id": 175  "dicom_field_name": "frames"}  {"dicom_field_expected_value": "1"  "dicom_file_value": "1.0"  "dicom_tag": "(0018  0050)"  "flag_value": 1  "dicom_file_id": 175  "dicom_field_name": "Slice Thickness"}  {"dicom_field_expected_value": "3200"  "dicom_file_value": "3200.0"  "dicom_tag": "(0018  0080)"  "flag_value": 1  "dicom_file_id": 175  "dicom_field_name": "Repetition Time"}  {"dicom_field_expected_value": "221"  "dicom_file_value": "221.0"  "dicom_tag": "(0018  0081)"  "flag_value": 1  "dicom_file_id": 175  "dicom_field_name": "E cho Time"}  {"dicom_field_expected_value": "750"  "dicom_file_value": "750.0"  "dicom_tag": "(0018  0095)"  "flag_value": 1  "dicom_file_id": 175  "dicom_field_name": "Pixel Bandwidth"}  {"dicom_field_expected_value": "[0  256  256  0 ]  "  "dicom_file_value": "[0  256  256  0 ]  "  "dicom_tag": "(0018  1310)"  "flag_value": 1  "dicom_file_id": 175  "dicom_field_name": "Acquisition Matrix"}  {"dicom_field_expected_value": "120"  "dicom_file_value": "120.0"  "dicom_tag": "(0018  1314)"  "flag_value": 1  "dicom_file_id": 175  "dicom_field_name": "Flip Angle"}  {"dicom_field_expected_value": "256"  "dicom_file_value": "256"  "dicom_tag": "(0028  0010)"  "flag_va  lue": 1  "dicom_file_id": 175  "dicom_field_name": "Rows"}  {"dicom_field_expected_value": "256"  "dicom_file_value": "256"  "dicom_tag": "(0028  0011)"  "flag_value": 1  "dicom_file_id": 175  "dicom_field_name": "Columns"}  {"dicom_field_expected_value": "['1'  '1' ]  "  "dicom_file_value": "['1'  '1' ]  "  "dicom_tag": "(0028  0030)"  "flag_value": 1  "dicom_file_id": 175  "dicom_field_name": "Pixel Spacing"} ]    "4": [{"dicom_field_expected_value": "192"  "dicom_file_value": "192"  "dicom_tag": ""  "flag_value": 1  "dicom_file_id": 188  "dicom_field_name": "frames"}  {"dicom_field_expected_value": "1"  "dicom_file_value": "1.0"  "dicom_tag": "(0018  0050)"  "flag_value": 1  "dicom_file_id": 188  "dicom_field_name": "Slice Thickness"}  {"dicom_field_expected_value": "6000"  "dicom_file_value": "6000.0"  "dicom_tag": "(0018  0080)"  "flag_value": 1  "dicom_file_id": 188  "dicom_field_name": "Repetition Time"}  {"dicom_field_expected_value": "263"  "dicom_file_value": "263.0"  "dicom_tag": "(0018  0081)"  "flag_value": 1  "dicom _file_id": 188  "dicom_field_name": "Echo Time"}  {"dicom_field_expected_value": "2100"  "dicom_file_value": "2100.0"  "dicom_tag": "(0018  0082)"  "flag_value": 1  "dicom_file_id": 188  "dicom_field_name": "Inversion Time"}  {"dicom_field_expected_value": "780"  "dicom_file_value": "780.0"  "dicom_tag": "(0018  0095)"  "flag_value": 1  "dicom_file_id": 188  "dicom_field_name": "Pixel Bandwidth"}  {"dicom_field_expected_value": "[0  256  256  0 ]  "  "dicom_file_value": "[0  256  256  0 ]  "  "dicom_tag": "(0018  1310)"  "flag_value": 1  "dicom_file_id": 188  "dicom_field_name": "Acquisition Matrix"}  {"dicom_field_expected_value": "120"  "dicom_file_value": "120.0"  "dicom_tag": "(0018  1314)"  "flag_value": 1  "dicom_file_id": 188  "dicom_field_name": "Flip Angle"}  {"dicom_field_expected_value": "256"  "dicom_file_value": "256"  "dicom_tag": "(0028  0010)"  "flag_value": 1  "dicom_file_id": 188  "dicom_field_name": "Rows"}  {"dicom_field_expected_value": "256"  "dicom_file_value": "256"  "dicom_tag":   "(0028  0011)"  "flag_value": 1  "dicom_file_id": 188  "dicom_field_name": "Columns"}  {"dicom_field_expected_value": "['1'  '1' ]  "  "dicom_file_value": "['1'  '1' ]  "  "dicom_tag": "(0028  0030)"  "flag_value": 1  "dicom_file_id": 188  "dicom_field_name": "Pixel Spacing"} ]    "9": [{"dicom_field_expected_value": "31"  "dicom_file_value": "31"  "dicom_tag": ""  "flag_value": 1  "dicom_file_id": 3  "dicom_field_name": "frames"}  {"dicom_field_expected_value": "2"  "dicom_file_value": "2.0"  "dicom_tag": "(0018  0050)"  "flag_value": 1  "dicom_file_id": 3  "dicom_field_name": "Slice Thickness"}  {"dicom_field_expected_value": "11600"  "dicom_file_value": "11600.0"  "dicom_tag": "(0018  0080)"  "flag_value": 1  "dicom_file_id": 3  "dicom_field_name": "Repetition Time"}  {"dicom_field_expected_value": "91"  "dicom_file_value": "91.0"  "dicom_tag": "(0018  0081)"  "flag_value": 1  "dicom_file_id": 3  "dicom_field_name": "Echo Time"}  {"dicom_field_expected_value": "1700"  "dicom_file_value": "1700.0"  "dicom_tag": "(00 18  0095)"  "flag_value": 1  "dicom_file_id": 3  "dicom_field_name": "Pixel Bandwidth"}  {"dicom_field_expected_value": "[128  0  0  128 ]  "  "dicom_file_value": "[128  0  0  128 ]  "  "dicom_tag": "(0018  1310)"  "flag_value": 1  "dicom_file_id": 3  "dicom_field_name": "Acquisition Matrix"}  {"dicom_field_expected_value": "90"  "dicom_file_value": "90.0"  "dicom_tag": "(0018  1314)"  "flag_value": 1  "dicom_file_id": 3  "dicom_field_name": "Flip Angle"}  {"dicom_field_expected_value": "1152"  "dicom_file_value": "1152"  "dicom_tag": "(0028  0010)"  "flag_value": 1  "dicom_file_id": 3  "dicom_field_name": "Rows"}  {"dicom_field_expected_value": "1152"  "dicom_file_value": "1152"  "dicom_tag": "(0028  0011)"  "flag_value": 1  "dicom_file_id": 3  "dicom_field_name": "Columns"}  {"dicom_field_expected_value": "['2'  '2' ]  "  "dicom_file_value": "['2'  '2' ]  "  "dicom_tag": "(0028  0030)"  "flag_value": 1  "dicom_file_id": 3  "dicom_field_name": "Pixel Spacing"} ]    "8": [{"dicom_field_expected_value": "72"  "dicom_file_va  lue": "72"  "dicom_tag": ""  "flag_value": 1  "dicom_file_id": 6  "dicom_field_name": "frames"}  {"dicom_field_expected_value": "2"  "dicom_file_value": "2.0"  "dicom_tag": "(0018  0050)"  "flag_value": 1  "dicom_file_id": 6  "dicom_field_name": "Slice Thickness"}  {"dicom_field_expected_value": "29"  "dicom_file_value": "29.0"  "dicom_tag": "(0018  0080)"  "flag_value": 1  "dicom_file_id": 6  "dicom_field_name": "Repetition Time"}  {"dicom_field_expected_value": "20"  "dicom_file_value": "20.0"  "dicom_tag": "(0018  0081)"  "flag_value": 1  "dicom_file_id": 6  "dicom_field_name": "Echo Time"}  {"dicom_field_expected_value": "120"  "dicom_file_value": "120.0"  "dicom_tag": "(0018  0095)"  "flag_value": 1  "dicom_file_id": 6  "dicom_field_name": "Pixel Bandwidth"}  {"dicom_field_expected_value": "[0  512  192  0 ]  "  "dicom_file_value": "[0  512  192  0 ]  "  "dicom_tag": "(0018  1310)"  "flag_value": 1  "dicom_file_id": 6  "dicom_field_name": "Acquisition Matrix"}  {"dicom_field_expected_value": "15"   "dicom_file_value": "15.0"  "dicom_tag": "(0018  1314)"  "flag_value": 1  "dicom_file_id": 6  "dicom_field_name": "Flip Angle"}  {"dicom_field_expected_value": "512"  "dicom_file_value": "512"  "dicom_tag": "(0028  0010)"  "flag_value": 1  "dicom_file_id": 6  "dicom_field_name": "Rows"}  {"dicom_field_expected_value": "384"  "dicom_file_value": "384"  "dicom_tag": "(0028  0011)"  "flag_value": 1  "dicom_file_id": 6  "dicom_field_name": "Columns"}  {"dicom_field_expected_value": "['0.5'  '0.5' ]  "  "dicom_file_value": "['0.5'  '0.5' ]  "  "dicom_tag": "(0028  0030)"  "flag_value": 1  "dicom_file_id": 6  "dicom_field_name": "Pixel Spacing"} ]  }  "smoothed_image_statistics": {"3": {"mean_diff": 15.87955379486084  "mean_std_ratio": 1.2886545223550259}  "2": {"mean_diff": -39.69376754760742  "mean_std_ratio": -0.8704012242099667}  "4": {"mean_diff": 0.3030291497707367  "mean_std_ratio": 0.016547458337000432}}  "cnr_calculation": {"11": {"cnr_value": 7.0779089925699}  "9": {"cnr_value": 6.821130872515915}  "3": {"cnr_value": 18.30396  5121999852}  "2": {"cnr_value": 25.377643727126117}  "4": {"cnr_value": 6.470757979856439}}  "gm_wm_statistics": {"3": {"gm_ratio": 3.9500966217120972  "wm_ratio": 7.4627726220470132}  "2": {"gm_ratio": 3.1271045929432577  "wm_ratio": 3.6450512683569283}  "4": {"gm_ratio": 2.2893568795712165  "wm_ratio": 3.01208420139415}}  "tbi_report": {"report_file": "preprocessing_report.pdf"}  "snr_calculation": {"11": {"snr_value": 221.60345607964334}  "9": {"snr_value": 222.04648692571962}  "3": {"snr_value": 64.65260322695484}  "2": {"snr_value": 125.9276028387245}  "4": {"snr_value": 53.83360541545809}}  "head_coverage": {"11": "GOOD"  "10": "GOOD"  "12": "GOOD"  "3": "GOOD"  "2": "GOOD"  "4": "GOOD"  "9": "GOOD"  "8": "GOOD"}  "registration": {"mni": {"11": "11_affine.txt"  "10": "10_affine.txt"  "12": "12_affine.txt"  "14": "14_affine.txt"  "3": "3_affine.txt"  "2": "2_affine.txt"  "4": "4_affine.txt"  "9": "9_affine.txt"  "8": "8_affine.txt"}  "main_modality": {"11": "3_to_11_affine.txt"  "10" : "3_to_10_affine.txt"  "12": "3_to_12_affine.txt"  "14": "3_to_14_affine.txt"  "2": "3_to_2_affine.txt"  "4": "3_to_4_affine.txt"  "9": "3_to_9_affine.txt"  "8": "3_to_8_affine.txt"}}  "tbi_fmri_qc": {"fmri_motion_detection": {"14": {"maximal_translation": [8.926260606575392  9.517926552514973  38.464529389719594 ]    "maximal_rotation": [4.515738139449345  0.699181574023182  0.9252113689914125 ]    "translation_figure": "14_translation.png"  "rotation_figure": "14_rotation.png"}}  "temporal_snr": {"14": {"temporal_snr_image": "14_temporal_snr.nii.gz"  "temporal_snr_value": 46.189703551279479  "standard_deviation_image": "14_temporal_snr.nii.gz"  "mean_image": "14_temporal_snr.nii.gz"}}  "percentage_signal_change": {"14": {"psc_mean": 0.35025587736313341  "pcs_std": 0.0013870050744161312}}  "fmri_snr_calculation": {"14": {"fmri_snr_value": 517.36715399094373}}}  "protocol_check": [["T2"  "PASS" ]    ["T1"  "PASS" ]    ["FLAIR"  "PASS" ]    ["SWI"  "PASS" ]    ["DTI"  "PASS" ]    ["DTI"  "PASS" ]    ["rs_fMRI"  "PASS" ]]  }'''
print(fix_xnat_json(badjson7))
json.loads(fix_xnat_json(badjson7))

In [ ]:
def JSON2DICTParser(data):
    # From https://stackoverflow.com/questions/20680272/parsing-a-json-string-which-was-loaded-from-a-csv-using-pandas
    if data and re.sub('[\[\]{}\s]+', '', data):
        try:
            j1 = json.loads(fix_xnat_json(data))
        except ValueError:
            print(data)
            raise
        return j1
    else:
        return float('nan')

# Load the csv into a Pandas DataFrame
sdf = pd.read_csv(subjects_filepath, converters={'software_metadata':JSON2DICTParser, 'parameters_dict':JSON2DICTParser, 'results_dict':JSON2DICTParser})
mdf = pd.read_csv(mrisessions_filepath, converters={'software_metadata':JSON2DICTParser, 'parameters_dict':JSON2DICTParser, 'results_dict':JSON2DICTParser})

In [ ]:
# Show structure of sdf
print('Columns for subjects dataframe: ')
print(sdf.columns)
sdf  # to pretty print, just put the dataframe name as the last line, without print or anything

In [ ]:
# Show structure of mdf
print('Columns for mri sessions dataframe: ')
print(mdf.columns)
mdf